```c
以下に課題の概要をまとめます。
課題概要: ft_irc

このプロジェクトでは、独自のIRCサーバーを作成します。実際のIRCクライアントを使用してサーバーに接続し、テストを行います。インターネットは、接続されたコンピュータが相互にやり取りするための堅固な標準プロトコルによって支配されています。これを知っておくことは常に良いことです。
目次

    Introduction
    General rules
    Mandatory Part
        Requirements
        For MacOS only
        Test example
    Bonus part
    Submission and peer-evaluation

Introduction

IRC（Internet Relay Chat）は、インターネット上のテキストベースの通信プロトコルです。リアルタイムのメッセージングを提供し、公開または非公開で行うことができます。ユーザーは直接メッセージを交換したり、グループチャネルに参加したりできます。IRCクライアントはIRCサーバーに接続してチャネルに参加します。IRCサーバーはネットワークを形成するために相互に接続されています。
General rules

    プログラムはどのような状況でもクラッシュしてはならず（メモリ不足の場合でも）、予期せず終了してはなりません。これが発生した場合、プロジェクトは機能しないと見なされ、評価は0になります。
    ソースファイルをコンパイルするためのMakefileを提出する必要があります。再リンクしてはなりません。
    Makefileには少なくとも以下のルールを含める必要があります: $(NAME), all, clean, fclean, re
    c++と-Wall -Wextra -Werrorフラグを使用してコードをコンパイルする必要があります。
    コードはC++ 98標準に準拠している必要があります。また、-std=c++98フラグを追加してもコンパイルできる必要があります。
    可能な限り最新のC++機能を使用して開発するように努めてください。

この概要を基に、課題の要件を完全に把握しました。次に、具体的な実装や改善点についての指示があればお知らせください。


```

```c
レビュー要項: ft_irc
Mandatory Part
Basic checks

    Makefileが存在し、プロジェクトが正しいオプションでコンパイルされ、C++で書かれており、実行ファイルが期待通りに呼び出されること。
    コード内にpoll()（または同等のもの）が1つだけ存在することを確認する。
    poll()（または同等のもの）が各accept、read/recv、write/sendの前に毎回呼び出されることを確認する。これらの呼び出し後、errnoを使用して特定のアクションをトリガーしないこと（例：errno == EAGAINの後に再度読み取るなど）。
    fcntl()の各呼び出しがfcntl(fd, F_SETFL, O_NONBLOCK);として行われていることを確認する。それ以外のfcntl()の使用は禁止されている。
    これらのポイントのいずれかが間違っている場合、評価は終了し、最終評価は0となる。

Networking

以下の要件を確認する：

    サーバーが起動し、コマンドラインから指定されたポートで全てのネットワークインターフェースでリスニングする。
    ncツールを使用してサーバーに接続し、コマンドを送信し、サーバーが応答することを確認する。
    チームに参照するIRCクライアントを尋ねる。
    このIRCクライアントを使用してサーバーに接続できることを確認する。
    サーバーが同時に複数の接続を処理できることを確認する。サーバーはブロックしてはならない。全ての要求に応答できること。IRCクライアントとncを同時に使用してテストする。
    適切なコマンドを使用してチャネルに参加する。1つのクライアントからの全てのメッセージがそのチャネルに参加している他の全てのクライアントに送信されることを確認する。

Networking specials

ネットワーク通信は多くの奇妙な状況によって妨げられる可能性がある。

    ncを使用して部分的なコマンドを送信する。サーバーが正しく応答することを確認する。部分的なコマンドが送信された状態で、他の接続が正常に動作することを確認する。
    クライアントを予期せず終了させる。その後、他の接続および新しいクライアントの接続が正常に動作することを確認する。
    コマンドの半分だけを送信した状態でncを予期せず終了させる。サーバーが異常な状態やブロックされていないことを確認する。
    チャネルに接続しているクライアントを停止（^-Z）させる。その後、別のクライアントを使用してチャネルをフラッドする。サーバーがハングしないことを確認する。クライアントが再びライブになったとき、全ての保存されたコマンドが正常に処理されることを確認する。また、この操作中にメモリリークがないことを確認する。

Client Commands basic

    ncおよび参照するIRCクライアントの両方を使用して、認証、ニックネームの設定、ユーザー名の設定、チャネルへの参加ができることを確認する。これが問題ないことを確認する（既にこれを行っているはず）。
    プライベートメッセージ（PRIVMSG）が異なるパラメータで完全に機能することを確認する。

Client Commands channel operator

    ncおよび参照するIRCクライアントの両方を使用して、通常のユーザーがチャネルオペレーターのアクションを行う権限を持たないことを確認する。その後、オペレーターでテストする。全てのチャネル操作コマンドをテストする（機能しない機能ごとに1ポイント減点）。

これらの要件を常に参考にしながら、実装を進めていきます。


```

---

了解しました。次のステップとして、各機能が正しく動作することを確認するためのテストを実施します。
ステップ7: エラーハンドリングとテスト
目標

各機能が正しく動作することを確認し、エラーハンドリングが適切に行われていることを確認します。
テストケース
テストケース1: クライアントの認証機能のテスト

    正しいパスワードで接続する
        クライアントが正しいパスワードを入力して接続できることを確認します。

    間違ったパスワードで接続する
        クライアントが間違ったパスワードを入力した場合に接続が拒否されることを確認します。

テストケース2: ニックネームとユーザー名の設定のテスト

    ニックネームを設定する
        クライアントがNICKコマンドを使用してニックネームを設定できることを確認します。

    ユーザー名を設定する
        クライアントがUSERコマンドを使用してユーザー名を設定できることを確認します。

テストケース3: チャネルの作成と参加のテスト

    チャネルを作成し、参加する
        クライアントがJOINコマンドを使用してチャネルを作成し、参加できることを確認します。

    既存のチャネルに参加する
        クライアントが既存のチャネルに参加できることを確認します。

テストケース4: メッセージの送受信のテスト

    チャネル内でメッセージを送信する
        クライアントがチャネル内でメッセージを送信し、他のクライアントがそのメッセージを受信できることを確認します。

テストケース5: プライベートメッセージの送信のテスト

    特定のクライアントにプライベートメッセージを送信する
        クライアントが特定の他のクライアントにプライベートメッセージを送信できることを確認します。

テストケース6: チャネルオペレーターの権限管理のテスト

    ユーザーをキックする
        チャネルオペレーターがKICKコマンドを使用してユーザーをキックできることを確認します。

    チャネルのモードを変更する
        チャネルオペレーターがMODEコマンドを使用してチャネルのモードを変更できることを確認します。

    ユーザーを招待する
        チャネルオペレーターがINVITEコマンドを使用してユーザーを招待できることを確認します。


In [ ]:
動作確認手順

    サーバーを起動する

クライアントを接続する

パスワードを入力する

ニックネームを設定する

ユーザー名を設定する

チャネルを作成し、参加する

チャネル内でメッセージを送信する

特定のクライアントにプライベートメッセージを送信する

チャネルオペレーターがユーザーをキックする

チャネルオペレーターがチャネルのモードを変更する

チャネルオペレーターがユーザーを招待する
